# **Air Quality**

---
---

# Data Pre-processing

In [12]:
import pandas as pd
#pip install pandas openpyxl (para leer excels)
import numpy as np
import zipfile
import folium
from pickle import dump
import pickle

In [2]:
pd.set_option('display.max_columns', None)

#### **Upload and view the polluting data for Spain in 2022**
- Concatenate the files to be able to work better with them.
- Process the data before the EDA.

In [3]:
C6H6_HH_22 = pd.read_csv('../data/raw/time_data_2022/C6H6_HH_2022.csv', sep = ';')
CO_HH_22 = pd.read_csv('../data/raw/time_data_2022/CO_HH_2022.csv', sep= ';')
NO2_HH_22 = pd.read_csv('../data/raw/time_data_2022/NO2_HH_2022.csv', sep = ';')
NOx_HH_22 = pd.read_csv('../data/raw/time_data_2022/NOx_HH_2022.csv', sep = ';')
O3_HH_22 = pd.read_csv('../data/raw/time_data_2022/O3_HH_2022.csv', sep = ';')
PM10_HH_22 = pd.read_csv('../data/raw/time_data_2022/PM10_HH_2022.csv', sep = ';')
PM25_HH_22 = pd.read_csv('../data/raw/time_data_2022/PM25_HH_2022.csv', sep = ';')
SO2_HH_22 = pd.read_csv('../data/raw/time_data_2022/SO2_HH_2022.csv', sep = ';')

In [4]:
pollutants = pd.concat([C6H6_HH_22, CO_HH_22, NO2_HH_22, NOx_HH_22, O3_HH_22, PM10_HH_22, PM25_HH_22, SO2_HH_22])
print(pollutants.shape)

(819614, 32)


In [5]:
pollutants.head()

PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD  PUNTO_MUESTREO  ANNO  MES  DIA  \
0          1         59         8        30  01059008_30_59  2022    1    1   
1          1         59         8        30  01059008_30_59  2022    1    3   
2          1         59         8        30  01059008_30_59  2022    1    4   
3          1         59         8        30  01059008_30_59  2022    1    5   
4          1         59         8        30  01059008_30_59  2022    1    6   

    H01   H02   H03   H04   H05   H06   H07   H08   H09   H10   H11   H12  \
0  1.58  0.73  0.55  0.68  0.65  0.72  0.56  0.59  0.56  0.84  0.87   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2  0.16  0.16  0.18  0.18  0.14  0.15  0.22  0.18  0.16  0.18  0.18  0.13   
3  0.23  0.26  0.22  0.20  0.21  0.23  0.28  0.31  0.34  0.40  0.35  0.29   
4  0.36  0.31  0.32  0.33  0.32  0.33  0.33  0.37  0.54  0.45  0.37  0.32   

    H13   H14   H15   H16   H17   H18   H19   H20   H21   H22   H23   H24  
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1   NaN  0.24  0.23  0.22  0.23  0.58  0.24  0.54  0.69  0.32  0.17  0.17  
2  0.17  0.11  0.17  0.18  0.19  0.25  0.25  0.28  0.25  0.20  0.20  0.22  
3  0.27  0.30  0.48  0.33  0.45  0.58  0.57  0.51  0.51  0.51  0.48  0.41  
4  0.30  0.28  0.30  0.26  0.29  0.36  0.43  0.32  0.32  0.30  0.30  0.28

In [6]:
# Crear una columna 'fecha' con el año, mes y día
pollutants['FECHA'] = pd.to_datetime(pollutants[['ANNO', 'MES', 'DIA']].astype(str).agg('-'.join, axis= 1), errors='coerce', format='%Y-%m-%d')
pollutants.drop(['ANNO', 'MES', 'DIA'], axis= 1, inplace= True)

pollutants.head()

PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD  PUNTO_MUESTREO   H01   H02   H03  \
0          1         59         8        30  01059008_30_59  1.58  0.73  0.55   
1          1         59         8        30  01059008_30_59   NaN   NaN   NaN   
2          1         59         8        30  01059008_30_59  0.16  0.16  0.18   
3          1         59         8        30  01059008_30_59  0.23  0.26  0.22   
4          1         59         8        30  01059008_30_59  0.36  0.31  0.32   

    H04   H05   H06   H07   H08   H09   H10   H11   H12   H13   H14   H15  \
0  0.68  0.65  0.72  0.56  0.59  0.56  0.84  0.87   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.24  0.23   
2  0.18  0.14  0.15  0.22  0.18  0.16  0.18  0.18  0.13  0.17  0.11  0.17   
3  0.20  0.21  0.23  0.28  0.31  0.34  0.40  0.35  0.29  0.27  0.30  0.48   
4  0.33  0.32  0.33  0.33  0.37  0.54  0.45  0.37  0.32  0.30  0.28  0.30   

    H16   H17   H18   H19   H20   H21   H22   H23   H24      FECHA  
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN 2022-01-01  
1  0.22  0.23  0.58  0.24  0.54  0.69  0.32  0.17  0.17 2022-01-03  
2  0.18  0.19  0.25  0.25  0.28  0.25  0.20  0.20  0.22 2022-01-04  
3  0.33  0.45  0.58  0.57  0.51  0.51  0.51  0.48  0.41 2022-01-05  
4  0.26  0.29  0.36  0.43  0.32  0.32  0.30  0.30  0.28 2022-01-06

In [7]:
measured_excel = pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name= 'Magnitudes')
measured_excel.head()

MAGNITUD                           NOMBRE ABREVIATURA UNIDAD  \
0         1                DIÓXIDO DE AZUFRE         SO2  µg/m3   
1         6              MONÓXIDO DE CARBONO          CO  mg/m3   
2         7            MONÓXIDO DE NITRÓGENO          NO  µg/m3   
3         8             DIÓXIDO DE NITRÓGENO         NO2  µg/m3   
4         9  PARTÍCULAS EN SUSPENSIÓN <2,5µM       PM2.5  µg/m3   

                               GRUPO  
0  Contaminantes con valor legislado  
1  Contaminantes con valor legislado  
2                                NaN  
3  Contaminantes con valor legislado  
4  Contaminantes con valor legislado

In [8]:
stations_excel= pd.read_excel('../data/raw/Metainformacion2022.xlsx', sheet_name='Estaciones')
stations_excel.head(3)

COD_LOCAL  PROVINCIA  MUNICIPIO  ESTACION COD_ESTACION_DEM  \
0    1022001          1         22         1          ES1672A   
1    1036004          1         36         4          ES1349A   
2    1051001          1         51         1          ES1544A   

             N_RED    NOMBRE  FECHA_INI FECHA_FIN  LATITUD_G  LONGITUD_G  \
0  CCAA País Vasco  EL CIEGO 2004-05-10       NaT   42.51833    -2.61944   
1  CCAA País Vasco    LLODIO 1994-01-01       NaT   43.14407    -2.96337   
2  CCAA País Vasco   AGURAIN 1998-01-01       NaT   42.84900    -2.39370   

   ALTITUD      N_CCAA N_PROVINCIA            N_MUNICIPIO TIPO_ESTACION  \
0      480  PAÍS VASCO       ÁLAVA                ELCIEGO       TRAFICO   
1      122  PAÍS VASCO       ÁLAVA                 LLODIO       TRAFICO   
2      594  PAÍS VASCO       ÁLAVA  SALVATIERRA O AGURAIN         FONDO   

   TIPO_AREA TIPO_SUBAREA_RURAL EST                    ZONA  \
0  SUBURBANA                NaN  ST             RESIDENCIAL   
1  SUBURBANA                NaN  ST   RESIDENCIAL/COMERCIAL   
2  SUBURBANA                NaN  SF  RESIDENCIAL/INDUSTRIAL   

                     DIRECCION  
0  C/GABRIEL CELAYA 8, ELCIEGO  
1               C/ LAMUZA, S/N  
2      CUARTEL DE LA ERTZANTZA

In [9]:
columnas_a_modificar = ['LATITUD_G', 'LONGITUD_G']

for columna in columnas_a_modificar:
    # Asegurarse de que todos los valores no-NaN sean strings y aplicar la operación de reemplazo
    stations_excel[columna] = stations_excel[columna].apply(lambda x: str(x).replace(',', '.') if pd.notnull(x) else x)

# Después de asegurarse de que todos los valores son strings y de reemplazar las comas
# Convertimos toda la columna a float
for columna in columnas_a_modificar:
    stations_excel[columna] = stations_excel[columna].astype(float)

stations_excel.head(3)

COD_LOCAL  PROVINCIA  MUNICIPIO  ESTACION COD_ESTACION_DEM  \
0    1022001          1         22         1          ES1672A   
1    1036004          1         36         4          ES1349A   
2    1051001          1         51         1          ES1544A   

             N_RED    NOMBRE  FECHA_INI FECHA_FIN  LATITUD_G  LONGITUD_G  \
0  CCAA País Vasco  EL CIEGO 2004-05-10       NaT   42.51833    -2.61944   
1  CCAA País Vasco    LLODIO 1994-01-01       NaT   43.14407    -2.96337   
2  CCAA País Vasco   AGURAIN 1998-01-01       NaT   42.84900    -2.39370   

   ALTITUD      N_CCAA N_PROVINCIA            N_MUNICIPIO TIPO_ESTACION  \
0      480  PAÍS VASCO       ÁLAVA                ELCIEGO       TRAFICO   
1      122  PAÍS VASCO       ÁLAVA                 LLODIO       TRAFICO   
2      594  PAÍS VASCO       ÁLAVA  SALVATIERRA O AGURAIN         FONDO   

   TIPO_AREA TIPO_SUBAREA_RURAL EST                    ZONA  \
0  SUBURBANA                NaN  ST             RESIDENCIAL   
1  SUBURBANA                NaN  ST   RESIDENCIAL/COMERCIAL   
2  SUBURBANA                NaN  SF  RESIDENCIAL/INDUSTRIAL   

                     DIRECCION  
0  C/GABRIEL CELAYA 8, ELCIEGO  
1               C/ LAMUZA, S/N  
2      CUARTEL DE LA ERTZANTZA

In [10]:
measured_dict = {}

for index, row in measured_excel.iterrows():
    measured_dict[row['MAGNITUD']] = row['ABREVIATURA']

measured_dict

{1: 'SO2',
 6: 'CO',
 7: 'NO',
 8: 'NO2',
 9: 'PM2.5',
 10: 'PM10',
 12: 'NOX',
 14: 'O3',
 17: 'As',
 19: 'Pb',
 27: 'BAP',
 28: 'Cd',
 30: 'C6H6',
 62: 'NI'}

In [11]:
# Crear diccionario de las magnitudes 
magnitud_dict = {
    1: 'SO2',
    6: 'CO',
    7: 'NO',
    8: 'NO2',
    9: 'PM2.5',
    10: 'PM10',
    12: 'NOX',
    14: 'O3',
    17: 'As',
    19: 'Pb',
    27: 'BAP',
    28: 'Cd',
    30: 'C6H6',
    62: 'NI'
}

In [15]:
# Crear el diccionario
dicc_latitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LATITUD_G'] for index, row in stations_excel.iterrows()}

In [16]:
# Crear el diccionario
dicc_longitude = {(row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION']): row['LONGITUD_G'] for index, row in stations_excel.iterrows()}

In [17]:
# Función para mapear cada fila al valor del diccionario
def mapear_latitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_latitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LATITUD'] = pollutants.apply(mapear_latitud, axis=1)

In [18]:
# Función para mapear cada fila al valor del diccionario
def mapear_longitud(row):
    # Crea una tupla con los valores de la fila
    clave = (row['PROVINCIA'], row['MUNICIPIO'], row['ESTACION'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_longitude.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['LONGITUD'] = pollutants.apply(mapear_longitud, axis=1)

In [19]:
pollutants.head(3)

PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD  PUNTO_MUESTREO   H01   H02   H03  \
0          1         59         8        30  01059008_30_59  1.58  0.73  0.55   
1          1         59         8        30  01059008_30_59   NaN   NaN   NaN   
2          1         59         8        30  01059008_30_59  0.16  0.16  0.18   

    H04   H05   H06   H07   H08   H09   H10   H11   H12   H13   H14   H15  \
0  0.68  0.65  0.72  0.56  0.59  0.56  0.84  0.87   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.24  0.23   
2  0.18  0.14  0.15  0.22  0.18  0.16  0.18  0.18  0.13  0.17  0.11  0.17   

    H16   H17   H18   H19   H20   H21   H22   H23   H24      FECHA  LATITUD  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN 2022-01-01  42.8548   
1  0.22  0.23  0.58  0.24  0.54  0.69  0.32  0.17  0.17 2022-01-03  42.8548   
2  0.18  0.19  0.25  0.25  0.28  0.25  0.20  0.20  0.22 2022-01-04  42.8548   

   LONGITUD  
0   -2.6807  
1   -2.6807  
2   -2.6807

In [20]:
# Dos claves son 'CLAVE1' y 'CLAVE2' y el valor es 'VALOR'
dicc_municipios = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_MUNICIPIO'] for index, row in stations_excel.iterrows()}
dicc_municipios

{(1, 22): 'ELCIEGO',
 (1, 36): 'LLODIO',
 (1, 51): 'SALVATIERRA O AGURAIN',
 (1, 55): 'VALDEGOVÍA',
 (1, 59): 'VITORIA-GASTEIZ',
 (2, 3): 'ALBACETE',
 (3, 2): 'AGOST',
 (3, 9): 'ALCOY/ALCOI',
 (3, 14): 'ALICANTE/ALACANT',
 (3, 31): 'BENIDORM',
 (3, 65): 'ELCHE/ELX',
 (3, 66): 'ELDA',
 (3, 99): 'ORIHUELA',
 (3, 105): 'PINOSO',
 (3, 133): 'TORREVIEJA',
 (4, 13): 'ALMERÍA',
 (4, 22): 'BÉDAR',
 (4, 24): 'BENAHADUX',
 (4, 32): 'CARBONERAS',
 (4, 35): 'CUEVAS DEL ALMANZORA',
 (4, 66): 'NÍJAR',
 (4, 902): 'EJIDO (EL)',
 (5, 19): 'ÁVILA',
 (6, 15): 'BADAJOZ',
 (6, 16): 'BARCARROTA',
 (6, 22): 'BURGUILLOS DEL CERRO',
 (6, 70): 'JEREZ DE LOS CABALLEROS',
 (6, 81): 'MEDINA DE LAS TORRES',
 (6, 83): 'MÉRIDA',
 (6, 158): 'ZAFRA',
 (7, 3): 'ALCÚDIA',
 (7, 10): 'BUNYOLA',
 (7, 15): 'CIUTADELLA DE MENORCA',
 (7, 19): 'ESCORCA',
 (7, 26): 'EIVISSA',
 (7, 32): 'MAHÓN',
 (7, 40): 'PALMA DE MALLORCA',
 (7, 42): 'POLLENÇA',
 (7, 44): 'POBLA (SA)',
 (7, 46): 'SANT ANTONI DE PORTMANY',
 (7, 54): 'SANTA EULAL

In [24]:
# Crear un diccionario vacío para los nombres de las provincias
dicc_provincias = {}

for index, row in stations_excel.iterrows():
    dicc_provincias[row['PROVINCIA']] = row['N_PROVINCIA']

dicc_provincias

{1: 'ÁLAVA',
 2: 'ALBACETE',
 3: 'ALICANTE/ALACANT',
 4: 'ALMERÍA',
 5: 'ÁVILA',
 6: 'BADAJOZ',
 7: 'BALEARS (ILLES)',
 8: 'BARCELONA',
 9: 'BURGOS',
 10: 'CÁCERES',
 11: 'CÁDIZ',
 12: 'CASTELLÓN/CASTELLÓ',
 13: 'CIUDAD REAL',
 14: 'CÓRDOBA',
 15: 'CORUÑA (A)',
 16: 'CUENCA',
 17: 'GIRONA',
 18: 'GRANADA',
 19: 'GUADALAJARA',
 20: 'GUIPÚZCOA',
 21: 'HUELVA',
 22: 'HUESCA',
 23: 'JAÉN',
 24: 'LEÓN',
 25: 'LLEIDA',
 26: 'RIOJA (LA)',
 27: 'LUGO',
 28: 'MADRID',
 29: 'MÁLAGA',
 30: 'MURCIA',
 31: 'NAVARRA',
 32: 'OURENSE',
 33: 'ASTURIAS',
 34: 'PALENCIA',
 35: 'PALMAS (LAS)',
 36: 'PONTEVEDRA',
 37: 'SALAMANCA',
 38: 'SANTA CRUZ DE TENERIFE',
 39: 'CANTABRIA',
 40: 'SEGOVIA',
 41: 'SEVILLA',
 42: 'SORIA',
 43: 'TARRAGONA',
 44: 'TERUEL',
 45: 'TOLEDO',
 46: 'VALENCIA',
 47: 'VALLADOLID',
 48: 'VIZCAYA',
 49: 'ZAMORA',
 50: 'ZARAGOZA',
 51: 'CEUTA'}

In [26]:
dict_ccaa = {(row['PROVINCIA'], row['MUNICIPIO']): row['N_CCAA'] 
                    for index, row in stations_excel.iterrows()}

dict_ccaa

{(1, 22): 'PAÍS VASCO',
 (1, 36): 'PAÍS VASCO',
 (1, 51): 'PAÍS VASCO',
 (1, 55): 'PAÍS VASCO',
 (1, 59): 'PAÍS VASCO',
 (2, 3): 'CASTILLA-LA MANCHA',
 (3, 2): 'COMUNIDAD VALENCIANA',
 (3, 9): 'COMUNIDAD VALENCIANA',
 (3, 14): 'COMUNIDAD VALENCIANA',
 (3, 31): 'COMUNIDAD VALENCIANA',
 (3, 65): 'COMUNIDAD VALENCIANA',
 (3, 66): 'COMUNIDAD VALENCIANA',
 (3, 99): 'COMUNIDAD VALENCIANA',
 (3, 105): 'COMUNIDAD VALENCIANA',
 (3, 133): 'COMUNIDAD VALENCIANA',
 (4, 13): 'ANDALUCÍA',
 (4, 22): 'ANDALUCÍA',
 (4, 24): 'ANDALUCÍA',
 (4, 32): 'ANDALUCÍA',
 (4, 35): 'ANDALUCÍA',
 (4, 66): 'ANDALUCÍA',
 (4, 902): 'ANDALUCÍA',
 (5, 19): 'CASTILLA Y LEÓN',
 (6, 15): 'EXTREMADURA',
 (6, 16): 'EXTREMADURA',
 (6, 22): 'EXTREMADURA',
 (6, 70): 'EXTREMADURA',
 (6, 81): 'EXTREMADURA',
 (6, 83): 'EXTREMADURA',
 (6, 158): 'EXTREMADURA',
 (7, 3): 'BALEARES (ISLAS)',
 (7, 10): 'BALEARES (ISLAS)',
 (7, 15): 'BALEARES (ISLAS)',
 (7, 19): 'BALEARES (ISLAS)',
 (7, 26): 'BALEARES (ISLAS)',
 (7, 32): 'BALEARES (ISLAS)

In [28]:
# Crear clave para agregar la CCAA
pollutants['CLAVE_COMBINADA'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)
pollutants['N_CCAA'] = pollutants['CLAVE_COMBINADA'].map(dict_ccaa)
pollutants.drop('CLAVE_COMBINADA', axis=1, inplace=True)

In [29]:
# Crear la clave combinada en el DataFrame objetivo.
pollutants['LOC_MUNIC'] = pollutants.apply(lambda x: (x['PROVINCIA'], x['MUNICIPIO']), axis=1)

# Usar el método .map() para mapear los nombres de municipio usando el diccionario.
pollutants['N_MUNICIPIO'] = pollutants['LOC_MUNIC'].map(dicc_municipios)

pollutants.drop('LOC_MUNIC', axis=1, inplace=True)

In [30]:
pollutants['MAGNITUD'] = pollutants['MAGNITUD'].map(measured_dict)
pollutants['PROVINCIA'] = pollutants['PROVINCIA'].map(dicc_provincias)
pollutants

PROVINCIA  MUNICIPIO  ESTACION MAGNITUD  PUNTO_MUESTREO   H01   H02  \
0          ÁLAVA         59         8     C6H6  01059008_30_59  1.58  0.73   
1          ÁLAVA         59         8     C6H6  01059008_30_59   NaN   NaN   
2          ÁLAVA         59         8     C6H6  01059008_30_59  0.16  0.16   
3          ÁLAVA         59         8     C6H6  01059008_30_59  0.23  0.26   
4          ÁLAVA         59         8     C6H6  01059008_30_59  0.36  0.31   
...          ...        ...       ...      ...             ...   ...   ...   
147190     CEUTA          1         1      SO2   51001001_1_38  1.64  1.50   
147191     CEUTA          1         1      SO2   51001001_1_38  2.30  2.15   
147192     CEUTA          1         1      SO2   51001001_1_38  1.40  1.22   
147193     CEUTA          1         1      SO2   51001001_1_38  1.68  1.40   
147194     CEUTA          1         1      SO2   51001001_1_38  3.86  3.54   

         H03   H04   H05   H06   H07   H08   H09   H10   H11   H12   H13  \
0       0.55  0.68  0.65  0.72  0.56  0.59  0.56  0.84  0.87   NaN   NaN   
1        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2       0.18  0.18  0.14  0.15  0.22  0.18  0.16  0.18  0.18  0.13  0.17   
3       0.22  0.20  0.21  0.23  0.28  0.31  0.34  0.40  0.35  0.29  0.27   
4       0.32  0.33  0.32  0.33  0.33  0.37  0.54  0.45  0.37  0.32  0.30   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
147190  1.47  1.44  1.58  1.38  1.12  1.22  1.46  1.18  1.37  2.75  8.84   
147191  2.17  2.05  1.82  1.74  1.85  2.48  2.71  3.53  3.34  2.98  2.99   
147192  1.22  1.11  1.04  1.16  1.15  1.23  1.18  1.43  1.65  1.91  2.09   
147193  1.52  1.66  1.53  1.57  1.48  1.40  2.78  2.35  2.17  2.26  2.35   
147194  3.87  4.46  4.78  4.35  4.37  4.73  6.99  7.67  5.39  4.06   NaN   

          H14    H15    H16    H17    H18    H19   H20    H21   H22   H23  \
0         NaN    NaN    NaN    NaN    NaN    NaN   NaN    NaN   NaN   NaN   
1        0.24   0.23   0.22   0.23   0.58   0.24  0.54   0.69  0.32  0.17   
2        0.11   0.17   0.18   0.19   0.25   0.25  0.28   0.25  0.20  0.20   
3        0.30   0.48   0.33   0.45   0.58   0.57  0.51   0.51  0.51  0.48   
4        0.28   0.30   0.26   0.29   0.36   0.43  0.32   0.32  0.30  0.30   
...       ...    ...    ...    ...    ...    ...   ...    ...   ...   ...   
147190  15.18  15.03  25.54  12.58  38.78  17.84  8.01  11.66  3.28  5.70   
147191   2.44   2.72   2.27   2.07   2.00   1.79  1.65   1.68  1.39  1.30   
147192   2.99   3.31   3.77   2.82   2.47   2.79  2.77   2.66  2.51  2.42   
147193   2.11   1.88   1.71   1.66   1.70   1.58  2.12   1.61  1.57  1.88   
147194    NaN   2.89    NaN   2.29   2.10   2.23  1.97   1.78  1.51   NaN   

         H24      FECHA  LATITUD  LONGITUD      N_CCAA      N_MUNICIPIO  
0        NaN 2022-01-01  42.8548   -2.6807  PAÍS VASCO  VITORIA-GASTEIZ  
1       0.17 2022-01-03  42.8548   -2.6807  PAÍS VASCO  VITORIA-GASTEIZ  
2       0.22 2022-01-04  42.8548   -2.6807  PAÍS VASCO  VITORIA-GASTEIZ  
3       0.41 2022-01-05  42.8548   -2.6807  PAÍS VASCO  VITORIA-GASTEIZ  
4       0.28 2022-01-06  42.8548   -2.6807  PAÍS VASCO  VITORIA-GASTEIZ  
...      ...        ...      ...       ...         ...              ...  
147190  5.62 2022-12-16  35.8930   -5.3147       CEUTA            CEUTA  
147191  1.28 2022-12-25  35.8930   -5.3147       CEUTA            CEUTA  
147192  2.52 2022-12-26  35.8930   -5.3147       CEUTA            CEUTA  
147193  1.62 2022-12-28  35.8930   -5.3147       CEUTA            CEUTA  
147194   NaN 2022-12-30  35.8930   -5.3147       CEUTA            CEUTA  

[819614 rows x 34 columns]

In [31]:
dicc_area = {(row['LATITUD_G'], row['LONGITUD_G']): row['TIPO_AREA'] for index, row in stations_excel.iterrows()}
dicc_area

{(42.51833, -2.61944): 'SUBURBANA',
 (43.14407, -2.96337): 'SUBURBANA',
 (42.849, -2.3937): 'SUBURBANA',
 (42.8752, -3.2317): 'RURAL',
 (42.8548, -2.6807): 'URBANA',
 (42.85607, -2.66779): 'URBANA',
 (42.84361, -2.66139): 'URBANA',
 (38.9808, -1.8452): 'SUBURBANA',
 (38.43606, -0.63804): 'URBANA',
 (38.70639, -0.46694): 'URBANA',
 (38.35944, -0.47194): 'URBANA',
 (38.34028, -0.50667): 'URBANA',
 (38.35111, -0.51389): 'SUBURBANA',
 (38.57139, -0.14667): 'SUBURBANA',
 (38.24222, -0.68278): 'SUBURBANA',
 (38.25917, -0.7175): 'URBANA',
 (38.45472, -0.80333): 'SUBURBANA',
 (38.08472, -0.85278): 'SUBURBANA',
 (38.45167, -1.06472): 'RURAL',
 (37.99111, -0.69): 'URBANA',
 (36.84133, -2.44672): 'URBANA',
 (36.86495, -2.3905): 'SUBURBANA',
 (37.19299, -1.97908): 'SUBURBANA',
 (36.9231, -2.46322): 'URBANA',
 (36.99678, -1.89535): 'URBANA',
 (37.25497, -1.78174): 'RURAL',
 (36.84745, -2.03819): 'SUBURBANA',
 (36.95, -1.96148): 'RURAL',
 (36.9581, -2.1592): 'SUBURBANA',
 (36.76972, -2.81097): 'URBA

In [34]:
dicc_tipoestacion = {(row['LATITUD_G'], row['LONGITUD_G']): row['TIPO_ESTACION'] for index, row in stations_excel.iterrows()}
dicc_tipoestacion

{(42.51833, -2.61944): 'TRAFICO',
 (43.14407, -2.96337): 'TRAFICO',
 (42.849, -2.3937): 'FONDO',
 (42.8752, -3.2317): 'FONDO',
 (42.8548, -2.6807): 'TRAFICO',
 (42.85607, -2.66779): 'TRAFICO',
 (42.84361, -2.66139): 'TRAFICO',
 (38.9808, -1.8452): 'FONDO',
 (38.43606, -0.63804): 'INDUSTRIAL',
 (38.70639, -0.46694): 'FONDO',
 (38.35944, -0.47194): 'TRAFICO',
 (38.34028, -0.50667): 'FONDO',
 (38.35111, -0.51389): 'INDUSTRIAL',
 (38.57139, -0.14667): 'FONDO',
 (38.24222, -0.68278): 'FONDO',
 (38.25917, -0.7175): 'TRAFICO',
 (38.45472, -0.80333): 'FONDO',
 (38.08472, -0.85278): 'FONDO',
 (38.45167, -1.06472): 'FONDO',
 (37.99111, -0.69): 'TRAFICO',
 (36.84133, -2.44672): 'TRAFICO',
 (36.86495, -2.3905): 'FONDO',
 (37.19299, -1.97908): 'FONDO',
 (36.9231, -2.46322): 'INDUSTRIAL',
 (36.99678, -1.89535): 'INDUSTRIAL',
 (37.25497, -1.78174): 'INDUSTRIAL',
 (36.84745, -2.03819): 'INDUSTRIAL',
 (36.95, -1.96148): 'INDUSTRIAL',
 (36.9581, -2.1592): 'INDUSTRIAL',
 (36.76972, -2.81097): 'FONDO',
 (

In [36]:
# Función para mapear cada fila al valor del diccionario
def mapear_area(row):
    # Crea una tupla con los valores de la fila
    clave = (row['LATITUD'], row['LONGITUD'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_area.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['TIPO_AREA'] = pollutants.apply(mapear_area, axis=1)

In [37]:
# Función para mapear cada fila al valor del diccionario
def mapear_estacion(row):
    # Crea una tupla con los valores de la fila
    clave = (row['LATITUD'], row['LONGITUD'])
    # Devuelve el valor correspondiente del diccionario
    return dicc_tipoestacion.get(clave, None)  # Retorna None si la clave no existe

# Aplica la función a cada fila y crea una nueva columna
pollutants['TIPO_ESTACION'] = pollutants.apply(mapear_estacion, axis=1)

In [38]:
# Nombres de las columnas de horas en orden
columnas_horas = ['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 
                  'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 
                  'H21', 'H22', 'H23', 'H24']

# Nuevo orden de las columnas
nuevo_orden = ['FECHA', 'N_CCAA', 'PROVINCIA', 'N_MUNICIPIO', 'ESTACION', 'MAGNITUD', 'TIPO_AREA', 'TIPO_ESTACION', 'LATITUD', 'LONGITUD'] + columnas_horas

pollutants = pollutants[nuevo_orden]
pollutants

FECHA      N_CCAA PROVINCIA      N_MUNICIPIO  ESTACION MAGNITUD  \
0      2022-01-01  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
1      2022-01-03  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
2      2022-01-04  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
3      2022-01-05  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
4      2022-01-06  PAÍS VASCO     ÁLAVA  VITORIA-GASTEIZ         8     C6H6   
...           ...         ...       ...              ...       ...      ...   
147190 2022-12-16       CEUTA     CEUTA            CEUTA         1      SO2   
147191 2022-12-25       CEUTA     CEUTA            CEUTA         1      SO2   
147192 2022-12-26       CEUTA     CEUTA            CEUTA         1      SO2   
147193 2022-12-28       CEUTA     CEUTA            CEUTA         1      SO2   
147194 2022-12-30       CEUTA     CEUTA            CEUTA         1      SO2   

       TIPO_AREA TIPO_ESTACION  LATITUD  LONGITUD   H01   H02   H03   H04  \
0         URBANA       TRAFICO  42.8548   -2.6807  1.58  0.73  0.55  0.68   
1         URBANA       TRAFICO  42.8548   -2.6807   NaN   NaN   NaN   NaN   
2         URBANA       TRAFICO  42.8548   -2.6807  0.16  0.16  0.18  0.18   
3         URBANA       TRAFICO  42.8548   -2.6807  0.23  0.26  0.22  0.20   
4         URBANA       TRAFICO  42.8548   -2.6807  0.36  0.31  0.32  0.33   
...          ...           ...      ...       ...   ...   ...   ...   ...   
147190    URBANA         FONDO  35.8930   -5.3147  1.64  1.50  1.47  1.44   
147191    URBANA         FONDO  35.8930   -5.3147  2.30  2.15  2.17  2.05   
147192    URBANA         FONDO  35.8930   -5.3147  1.40  1.22  1.22  1.11   
147193    URBANA         FONDO  35.8930   -5.3147  1.68  1.40  1.52  1.66   
147194    URBANA         FONDO  35.8930   -5.3147  3.86  3.54  3.87  4.46   

         H05   H06   H07   H08   H09   H10   H11   H12   H13    H14    H15  \
0       0.65  0.72  0.56  0.59  0.56  0.84  0.87   NaN   NaN    NaN    NaN   
1        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   0.24   0.23   
2       0.14  0.15  0.22  0.18  0.16  0.18  0.18  0.13  0.17   0.11   0.17   
3       0.21  0.23  0.28  0.31  0.34  0.40  0.35  0.29  0.27   0.30   0.48   
4       0.32  0.33  0.33  0.37  0.54  0.45  0.37  0.32  0.30   0.28   0.30   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...    ...    ...   
147190  1.58  1.38  1.12  1.22  1.46  1.18  1.37  2.75  8.84  15.18  15.03   
147191  1.82  1.74  1.85  2.48  2.71  3.53  3.34  2.98  2.99   2.44   2.72   
147192  1.04  1.16  1.15  1.23  1.18  1.43  1.65  1.91  2.09   2.99   3.31   
147193  1.53  1.57  1.48  1.40  2.78  2.35  2.17  2.26  2.35   2.11   1.88   
147194  4.78  4.35  4.37  4.73  6.99  7.67  5.39  4.06   NaN    NaN   2.89   

          H16    H17    H18    H19   H20    H21   H22   H23   H24  
0         NaN    NaN    NaN    NaN   NaN    NaN   NaN   NaN   NaN  
1        0.22   0.23   0.58   0.24  0.54   0.69  0.32  0.17  0.17  
2        0.18   0.19   0.25   0.25  0.28   0.25  0.20  0.20  0.22  
3        0.33   0.45   0.58   0.57  0.51   0.51  0.51  0.48  0.41  
4        0.26   0.29   0.36   0.43  0.32   0.32  0.30  0.30  0.28  
...       ...    ...    ...    ...   ...    ...   ...   ...   ...  
147190  25.54  12.58  38.78  17.84  8.01  11.66  3.28  5.70  5.62  
147191   2.27   2.07   2.00   1.79  1.65   1.68  1.39  1.30  1.28  
147192   3.77   2.82   2.47   2.79  2.77   2.66  2.51  2.42  2.52  
147193   1.71   1.66   1.70   1.58  2.12   1.61  1.57  1.88  1.62  
147194    NaN   2.29   2.10   2.23  1.97   1.78  1.51   NaN   NaN  

[819614 rows x 34 columns]

In [39]:
pollutants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 819614 entries, 0 to 147194
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   FECHA          819614 non-null  datetime64[ns]
 1   N_CCAA         818524 non-null  object        
 2   PROVINCIA      819614 non-null  object        
 3   N_MUNICIPIO    818524 non-null  object        
 4   ESTACION       819614 non-null  int64         
 5   MAGNITUD       819614 non-null  object        
 6   TIPO_AREA      815658 non-null  object        
 7   TIPO_ESTACION  815658 non-null  object        
 8   LATITUD        815658 non-null  float64       
 9   LONGITUD       815658 non-null  float64       
 10  H01            809146 non-null  float64       
 11  H02            809010 non-null  float64       
 12  H03            807769 non-null  float64       
 13  H04            808729 non-null  float64       
 14  H05            809302 non-null  float64       
 15  H06  

In [40]:
total_nan = pollutants.isna().sum().sum()
total_nan

385599

In [41]:
pollutants.dropna(inplace= True)
pollutants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715747 entries, 2 to 147193
Data columns (total 34 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   FECHA          715747 non-null  datetime64[ns]
 1   N_CCAA         715747 non-null  object        
 2   PROVINCIA      715747 non-null  object        
 3   N_MUNICIPIO    715747 non-null  object        
 4   ESTACION       715747 non-null  int64         
 5   MAGNITUD       715747 non-null  object        
 6   TIPO_AREA      715747 non-null  object        
 7   TIPO_ESTACION  715747 non-null  object        
 8   LATITUD        715747 non-null  float64       
 9   LONGITUD       715747 non-null  float64       
 10  H01            715747 non-null  float64       
 11  H02            715747 non-null  float64       
 12  H03            715747 non-null  float64       
 13  H04            715747 non-null  float64       
 14  H05            715747 non-null  float64       
 15  H06  

/tmp/ipykernel_2095/1091851545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pollutants.dropna(inplace= True)


**Because the dataset has a large number of values, we are going to divide the dataset into subsets to facilitate the operation of the machine when working with the data.**
- We will first save the entire dataset so we can return to it if we need to extract more data in the future.
- We will save the community of interest with which we are going to work in a separate .csv.

In [42]:
pollutants.to_csv('../data/interim/pollutants.csv', index= False)
pollutants = pd.read_csv('../data/interim/pollutants.csv')

In [43]:
madrid_df = pollutants[pollutants['N_CCAA'] == 'MADRID']
madrid_df.head()

FECHA  N_CCAA PROVINCIA N_MUNICIPIO  ESTACION MAGNITUD TIPO_AREA  \
10386  2022-01-01  MADRID    MADRID  ALCOBENDAS         4     C6H6    URBANA   
10387  2022-01-02  MADRID    MADRID  ALCOBENDAS         4     C6H6    URBANA   
10388  2022-01-03  MADRID    MADRID  ALCOBENDAS         4     C6H6    URBANA   
10389  2022-01-04  MADRID    MADRID  ALCOBENDAS         4     C6H6    URBANA   
10390  2022-01-05  MADRID    MADRID  ALCOBENDAS         4     C6H6    URBANA   

      TIPO_ESTACION   LATITUD  LONGITUD  H01  H02  H03  H04  H05  H06  H07  \
10386       TRAFICO  40.54035  -3.64525  1.8  1.8  1.6  1.3  1.2  1.0  0.9   
10387       TRAFICO  40.54035  -3.64525  1.2  1.2  1.0  0.9  0.9  0.8  0.8   
10388       TRAFICO  40.54035  -3.64525  1.5  1.3  1.1  1.0  1.0  0.9  1.0   
10389       TRAFICO  40.54035  -3.64525  1.8  1.6  1.5  1.6  1.5  1.4  3.5   
10390       TRAFICO  40.54035  -3.64525  0.3  0.3  0.3  0.3  0.3  0.3  0.3   

       H08  H09  H10  H11  H12  H13  H14  H15  H16  H17  H18  H19  H20  H21  \
10386  0.8  0.9  0.7  0.5  0.5  0.5  0.6  0.5  0.4  0.3  0.5  0.7  1.5  1.9   
10387  0.8  1.0  0.7  0.7  0.6  0.6  0.4  0.5  0.8  0.5  0.9  1.2  1.5  1.9   
10388  1.2  2.3  1.8  1.3  0.8  0.5  0.8  0.8  0.9  0.9  1.2  0.9  0.8  1.1   
10389  3.1  1.1  1.2  1.2  1.2  0.8  0.7  0.8  0.6  0.3  0.3  0.3  0.3  0.3   
10390  0.4  0.9  0.5  0.5  0.4  0.5  0.5  0.4  0.3  0.4  0.4  0.4  0.4  0.4   

       H22  H23  H24  
10386  2.0  1.7  1.5  
10387  1.2  1.6  1.5  
10388  1.5  1.5  2.1  
10389  0.3  0.4  0.3  
10390  0.7  0.6  0.5

In [36]:
madrid_df.to_csv('../data/processed/Madrid_pollutants.csv', index= False)

**Map of all the communities where the values have been recorded.**
- To view all the stations where pollutant values have been recorded.

In [39]:
# Crear un mapa base
m = folium.Map(location=[stations_excel['LATITUD_G'].mean(), stations_excel['LONGITUD_G'].mean()], zoom_start=6)

# Añadir puntos al mapa
for idx, row in stations_excel.iterrows():
    folium.Marker(location=[row['LATITUD_G'], row['LONGITUD_G']],
                  popup=row['N_MUNICIPIO'] if 'NombrePunto' in stations_excel.columns else "Sin Nombre").add_to(m)

# Guardar el mapa en un archivo HTML
m.save('mapa_CCAA.html')
#Aqui no se ve nada, creo que hay qeu hacerlo en un navegador normal